In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from statsmodels.tsa.api import VAR
from sklearn.metrics import mean_squared_error, r2_score
import os
from dotenv import load_dotenv
import yfinance as yf
import pandas as pd
import pandas_ta as ta
from scikeras.wrappers import KerasRegressor
# Load environment variables from the .env file
load_dotenv('../.env');

# Data Viz. 
import statsmodels.formula.api as smf
from statsmodels.tsa.seasonal import seasonal_decompose
from scipy.ndimage import gaussian_filter
from calendar import monthrange
from calendar import month_name

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
from sklearn.metrics import accuracy_score, confusion_matrix
import requests
import csv
from itertools import permutations
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error
from math import sqrt
from statsmodels.tsa.stattools import adfuller,kpss
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.graphics.tsaplots import plot_pacf

import statsmodels.graphics.tsaplots as tsaplot
from statsmodels.tsa.holtwinters import Holt, ExponentialSmoothing, SimpleExpSmoothing

import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
import yfinance as yf
from ta import add_all_ta_features
from sklearn.linear_model import LinearRegression

def load_and_prepare(ticker, period="max"):
    # Fetch historical stock data
    df = yf.download(ticker, period=period)

    # Calculate daily returns
    df['returns'] = df['Adj Close'].pct_change()

    # Fetch market data (e.g., S&P 500)
    market_data = yf.download('^GSPC', start=df.index.min(), end=df.index.max())
    market_data['market_returns'] = market_data['Adj Close'].pct_change()

    # Combine stock and market data
    merged_data = pd.merge(df, market_data[['market_returns']], left_index=True, right_index=True, how='inner')

    # Drop rows with missing values
    merged_data.dropna(inplace=True)

    # Initialize lists to store alpha and beta values
    alpha_values = []
    beta_values = []

    # Set up X and y for linear regression
    X = merged_data['market_returns'].values.reshape(-1, 1)
    y = merged_data['returns'].values

    # Iterate through the data to calculate alpha and beta for each day
    for i in range(len(merged_data)):
        X_i = X[:i + 1]
        y_i = y[:i + 1]

        # Fit linear regression model
        model = LinearRegression()
        model.fit(X_i, y_i)

        # Extract alpha and beta for the current day
        alpha_i = model.intercept_
        beta_i = model.coef_[0]

        alpha_values.append(alpha_i)
        beta_values.append(beta_i)

    # Add alpha and beta columns to the DataFrame
    merged_data['alpha'] = alpha_values
    merged_data['beta'] = beta_values

    # Drop columns not needed for the final result
    merged_data.drop(['returns', 'market_returns'], axis=1, inplace=True)

    # Add technical analysis features
    merged_data = add_all_ta_features(merged_data, open="Open", high="High", low="Low", close="Close", volume="Volume", fillna=True)

    # Calculate target and target class
    merged_data['target'] = ((merged_data['Close'] - merged_data['Open']) / merged_data['Open']) * 100
    merged_data['target'] = merged_data['target'].shift(-1)
    merged_data['target_class'] = np.select(
        [(merged_data['target'] > 1), (merged_data['target'] < -1), ((merged_data['target'] >= -1) & (merged_data['target'] <= 1))],
        [2, 0, 1],
        default=np.nan
    )
    merged_data['target_next_close'] = merged_data['Close'].shift(-1)

    # Drop rows with missing values
    merged_data.dropna(inplace=True)

    return merged_data

# Example usage:
ticker_symbol = 'AAPL'
stock_data = load_and_prepare(ticker_symbol)
stock_data.head()


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume,alpha,beta,volume_adi,volume_obv,...,momentum_roc,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,others_dr,others_dlr,others_cr,target,target_class,target_next_close
Date,,,,,,,,,,,,,,,,,,,,,
1980-12-15,0.122210,0.122210,0.121652,0.121652,0.094137,175884800,-0.052170,0.000000,-175884800.0,175884800,...,0.0,0.000000,0.000000,0.000000,-99.396685,0.000000,0.000000,-0.492577,1.0,0.112723
1980-12-16,0.113281,0.113281,0.112723,0.112723,0.087228,105728000,-0.047138,-2.955906,-281612800.0,70156800,...,0.0,-3.278821,-0.655764,-2.623057,-7.339788,-7.623102,-7.339788,0.000000,1.0,0.115513
1980-12-17,0.115513,0.116071,0.115513,0.115513,0.089387,86441600,-0.081198,5.077265,-368054400.0,156598400,...,0.0,-6.966390,-1.917889,-5.048501,2.475091,2.444957,-5.046364,0.000000,1.0,0.118862
1980-12-18,0.118862,0.119420,0.118862,0.118862,0.091978,73449600,-0.027458,1.312919,-441504000.0,230048000,...,0.0,-10.749482,-3.684208,-7.065274,2.899246,2.858013,-2.293424,0.000000,1.0,0.126116
1980-12-19,0.126116,0.126674,0.126116,0.126116,0.097592,48630400,-0.006341,0.611175,-490134400.0,278678400,...,0.0,-15.402172,-6.027801,-9.374371,6.102867,5.923888,3.669478,0.000000,1.0,0.132254


In [3]:
stock_data.columns

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'alpha', 'beta',
       'volume_adi', 'volume_obv', 'volume_cmf', 'volume_fi', 'volume_mfi',
       'volume_em', 'volume_sma_em', 'volume_vpt', 'volume_nvi', 'volume_vwap',
       'volatility_atr', 'volatility_bbm', 'volatility_bbh', 'volatility_bbl',
       'volatility_bbw', 'volatility_bbp', 'volatility_bbhi',
       'volatility_bbli', 'volatility_kcc', 'volatility_kch', 'volatility_kcl',
       'volatility_kcw', 'volatility_kcp', 'volatility_kchi',
       'volatility_kcli', 'volatility_dcl', 'volatility_dch', 'volatility_dcm',
       'volatility_dcw', 'volatility_dcp', 'volatility_ui', 'trend_macd',
       'trend_macd_signal', 'trend_macd_diff', 'trend_sma_fast',
       'trend_sma_slow', 'trend_ema_fast', 'trend_ema_slow', 'trend_adx',
       'trend_adx_pos', 'trend_adx_neg', 'trend_vortex_ind_pos',
       'trend_vortex_ind_neg', 'trend_vortex_ind_diff', 'trend_trix',
       'trend_mass_index', 'trend_cci', 'trend_dpo', '

In [6]:
df = stock_data

In [7]:
df.isna().sum()

Open                 0
High                 0
Low                  0
Close                0
Adj Close            0
                    ..
others_dlr           0
others_cr            0
target               0
target_class         0
target_next_close    0
Length: 94, dtype: int64

In [11]:
from sklearn.model_selection import train_test_split

# Assuming X is your feature set and y is your target variable
X = df.drop(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume',
       'volatility_atr', 'volatility_bbm', 'volatility_bbh', 'volatility_bbl',
       'volatility_bbw', 'volatility_bbp', 'volatility_bbhi',
       'volatility_bbli', 'volatility_kcc', 'volatility_kch', 'volatility_kcl',
       'volatility_kcw', 'volatility_kcp', 'volatility_kchi',
       'volatility_kcli', 'volatility_dcl', 'volatility_dch', 'volatility_dcm',
       'volatility_dcw', 'volatility_dcp', 'volatility_ui',
       'trend_adx',
       'trend_adx_pos', 'trend_adx_neg', 'trend_vortex_ind_pos',
       'trend_vortex_ind_neg', 'trend_vortex_ind_diff', 'trend_trix',
       'trend_mass_index', 'trend_cci', 'trend_dpo', 'trend_kst',
       'trend_kst_sig', 'trend_kst_diff', 'trend_ichimoku_conv',
       'trend_ichimoku_base', 'trend_ichimoku_a', 'trend_ichimoku_b',
       'trend_visual_ichimoku_a', 'trend_visual_ichimoku_b', 'trend_aroon_up',
       'trend_aroon_down', 'trend_aroon_ind', 'trend_psar_up',
       'trend_psar_down', 'trend_psar_up_indicator',
       'trend_psar_down_indicator', 'trend_stc', 'momentum_rsi',
       'momentum_stoch_rsi', 'momentum_stoch_rsi_k', 'momentum_stoch_rsi_d',
       'momentum_tsi', 'momentum_uo', 'momentum_stoch',
       'momentum_stoch_signal', 'momentum_wr', 'momentum_ao', 'momentum_kama',
       'momentum_roc', 'momentum_ppo', 'momentum_ppo_signal',
       'momentum_ppo_hist', 'others_dr', 'others_dlr', 'others_cr', 'target',
       'target_class', 'target_next_close'], axis=1)# Adjust columns accordingly
y = df[['target_class']]  # Adjust the target variable accordingly

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,stratify=y, random_state=42)

# Create a preprocessing pipeline
preprocessing_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', MinMaxScaler(feature_range=(0,1)))
])

# Apply the preprocessing pipeline to numeric features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', preprocessing_pipeline, X_train.columns)
    ])


# Fit and transform the training data
train_features_preprocessed = preprocessor.fit_transform(X_train)

# Transform the test data using the learned parameters from the training data
test_features_preprocessed = preprocessor.transform(X_test)

# train_features_preprocessed and test_features_preprocessed are preprocessed data

# Reshape training data
X_train_lstm = train_features_preprocessed.reshape((train_features_preprocessed.shape[0], 1, train_features_preprocessed.shape[1]))

# Reshape testing data
X_test_lstm = test_features_preprocessed.reshape((test_features_preprocessed.shape[0], 1, test_features_preprocessed.shape[1]))

model = Sequential()
model.add(LSTM(units=50, activation='relu', input_shape=(X_train_lstm.shape[1], X_train_lstm.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(units=3, activation='softmax'))  # 3 units for your three categories
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Fit the model
history = model.fit(X_train_lstm, y_train, epochs=50, batch_size=32, validation_split=0.2)

evaluation = model.evaluate(X_test_lstm, y_test)
print(f"Test Loss: {evaluation[0]}, Test Accuracy: {evaluation[1]}")


Epoch 1/50
218/218 [==============================] - 9s 28ms/step - loss: 1.0548 - accuracy: 0.4623 - val_loss: 1.0514 - val_accuracy: 0.4577
Epoch 2/50
218/218 [==============================] - 3s 13ms/step - loss: 1.0429 - accuracy: 0.4692 - val_loss: 1.0432 - val_accuracy: 0.4577
Epoch 3/50
218/218 [==============================] - 3s 13ms/step - loss: 1.0353 - accuracy: 0.4715 - val_loss: 1.0372 - val_accuracy: 0.4686
Epoch 4/50
218/218 [==============================] - 3s 13ms/step - loss: 1.0303 - accuracy: 0.4734 - val_loss: 1.0294 - val_accuracy: 0.4652
Epoch 5/50
218/218 [==============================] - 3s 12ms/step - loss: 1.0265 - accuracy: 0.4701 - val_loss: 1.0270 - val_accuracy: 0.4646
Epoch 6/50
218/218 [==============================] - 3s 13ms/step - loss: 1.0254 - accuracy: 0.4679 - val_loss: 1.0250 - val_accuracy: 0.4640
Epoch 7/50
218/218 [==============================] - 3s 13ms/step - loss: 1.0233 - accuracy: 0.4660 - val_loss: 1.0238 - val_accuracy: 0.4611